In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_path = './data/dataset.csv'
para_path = './data/wind_turbine_parameters.csv'

## 读入数据

In [ ]:
raw_data = pd.read_csv(data_path)
para = pd.read_csv(para_path)

In [ ]:
raw_data.head()

In [ ]:
para.head()

## 预处理部分数据

### 去掉标号的‘#’

In [ ]:
def remove_chr(row):
    return row.iloc[0][:-1]
para['Number']=para[['Number']].apply(remove_chr, axis=1)

In [ ]:
para.head()

### 将WheelSpeedRange 切分为WheelSpeedMin和WheelSpeedMax

In [ ]:
def get_min_wheel_speed(row):
    return float(row.iloc[0].split('-')[0])
def get_max_wheel_speed(row):
    return float(row.iloc[0].split('-')[1])
para['WheelSpeedMin'] = para[['WheelSpeedRange']].apply(get_min_wheel_speed, axis=1)
para['WheelSpesdMax'] = para[['WheelSpeedRange']].apply(get_max_wheel_speed, axis=1)

In [ ]:
para.columns.values

In [ ]:
para = para[['Number', 'Width', 'RatedPower', 'CutInWindSpeed',
       'CutOutWindSpeed', 'WheelSpeedMin',
       'WheelSpesdMax']]

In [ ]:
para.head()

In [ ]:
raw_data.head()

## 合并raw data和para data

In [ ]:
def insert_para(row):
    num = row['WindNumber']
    para_ = para.iloc[num-1,1:]
    return pd.concat([row, para_])
combined_data = raw_data.apply(insert_para, axis=1)

In [ ]:
combined_data.head()

In [ ]:
raw_data.info()

In [ ]:
 combined_data.info()

- 发现数据中没有缺失值

## 保存数据

In [ ]:
combined_data.to_csv('./data/combined_data.csv', index=0)

## 导入新数据

In [ ]:
combined_data = pd.read_csv('./data/combined_data.csv')
combined_data.head()

## 可视化数据

### Time-WindSpeed-Power

- 数据按照编号分组

In [ ]:
group_data = combined_data.groupby('WindNumber')

- 查看每组数据量

In [ ]:
rotor_cnt = group_data['WindNumber'].count()
type(rotor_cnt), rotor_cnt.values.shape,rotor_cnt, rotor_cnt.values

- 无法直接根据`Time`或者时间对应的index可视化所有数据，从[dataset](data/dataset.csv)中可以发现不同风电机的运行时间不同：

| Number | Start Time     | End Time          |
| ------ | -------------- | ----------------- |
| 1      | 2017/11/1 0:20 | 2017/11/1 0:20    |
| 2      | 2017/11/1 0:10 | 2017/11/1 0:10    |
| 3      | 2017/11/1 0:10 | 2018/10/30 23:500 |
| 4      | 2018/11/1 0:00 | 2019/10/17 14:50  |
| 5      | 2019/1/2 11:40 | 2017/11/1 0:10    |


- 如下的方式无法使用，可将Time映射到index，或者直接使用range

`sns.relplot(x='Time' , y='WindSpeed', hue='WindNumber',kind='line',data=combined_data)`

In [ ]:
group_data.get_group(1)

- 使用g_idx控制绘图数据

In [ ]:
g_idx = 0
data_size = 10000

- 绘制wind_speed-power图

In [ ]:
# 查看：5000数据
g_data = group_data.get_group(g_idx+1).iloc[:5000]
sns.relplot(x='WindSpeed' , y='Power', kind='scatter', data=g_data, height=4, aspect=4)

- 前5000条数据，并没有出现曲线左上方的异常

In [ ]:
# 查看5000：10000的数据
data_size=5000
g_data = group_data.get_group(g_idx+1).iloc[data_size:data_size*2]
sns.relplot(x='WindSpeed' , y='Power', kind='scatter', data=g_data, height=5, aspect=4)

- 从5000-10000开始有了左上方的异常，以及曲线右边的异常点

- 看到后面很多在wind_speed很大的时候，power降到了0,这是异常
- 切入风速以下，应该Power为0，否则，也可能是异常
- 有部分数据在风速上升的时候保持了不变，应该是达到了额定功率

In [ ]:
# 查看所有数据
g_data = group_data.get_group(g_idx+1)
plt.figure(figsize=(25,10))
plt.plot(g_data['WindSpeed'], g_data['Power'], '.b', ms=4, label='WindSpeed-Power')
plt.legend()
plt.xlabel('Wind Speed')
plt.ylabel('Power')
plt.xticks(np.linspace(0, int(max(g_data['WindSpeed'])), int(max(g_data['WindSpeed']))+1))

In [ ]:
wind_speed = g_data['WindSpeed']
power = g_data['Power']
time = np.arange(power.shape[0])

plt.figure(figsize=(25,10))
ax=plt.subplot(111)
ax.plot(time, wind_speed, 'b-', lw=.5, label='wind_speed')

ax.set_xlabel('time')
ax.set_ylabel('wind_speed')
plt.legend(loc='upper left')

# 重点来了，twinx 或者 twiny 函数
ax2 = ax.twinx()
ax2.plot(time, power, '.r', ms=2, label='power')

ax2.set_ylabel('power')
plt.title('wind_speed and power')
plt.legend(loc='upper right')

- 这里看的也很明显，红色power曲线下方有很多为0的部分，而蓝色线非0

- 有风但是功率为0的可能有很多，比如断电了，坏了，之类的

- 下面查看有风时电机转不转，数据同上面一样为最后的5k条数据

### WindSpeed-RotorSpeed

- 底部存在有风的时候，电机速度为0,说明是很多是不转了导致power为0
- 上方存在有风但是电机转的特别快的点
- 风速和电机曲线右侧，偏低的点
- 风速和电机曲线右侧，呈一条直线的点

- 查看windspeed-power-rotorspeed关系

In [ ]:
g_idx = 0
g_data = group_data.get_group(g_idx+1)

wind_speed = g_data['WindSpeed']
rotor_speed = g_data['RotorSpeed']

plt.figure(figsize=(25,10))
ax=plt.subplot(111)
ax.plot(wind_speed, rotor_speed, '.r', ms=2, label='power')
ax.set_xlabel('wind speed')
ax.set_ylabel('RotorSpeed')
plt.legend(loc='upper right')

### wind speed - rotor speed - power

In [ ]:
wind_speed = g_data['WindSpeed']
power = g_data['Power']
rotor_speed = g_data['RotorSpeed']

plt.figure(figsize=(25,10))
ax=plt.subplot(111)
ax.plot(wind_speed, power, '.b', ms=2, label='power')
ax.set_xlabel('wind speed')
ax.set_ylabel('power')
plt.legend(loc='upper left')

ax2 = ax.twinx()
ax2.plot(wind_speed, rotor_speed, 'vr', ms=2, label='rotor speed')
ax2.set_ylabel('rotor speed')
plt.title('wind_speed, power and rotor speed')
plt.legend(loc='upper right')

1. wind speed在2.5-5范围内，随着wind spee增大，power和rotor speed增加并不同步，有很多点在中间部位
1. wind speed 在cut in speed 之前，rotor speed和power应该都是0，但是有很多明显的异常点
1. 后半部分，有很多是wind speed高，但是rotor speed和power都几乎为0，这应该可以说明是**有风不转**
1. 右半部分，中间区域有很多零散点

### rotor speed - power

In [ ]:
g_idx = 0
g_data = group_data.get_group(g_idx+1)
power = g_data['Power']
rotor_speed = g_data['RotorSpeed']

plt.figure(figsize=(25,10))
ax=plt.subplot(111)
ax.plot(rotor_speed, power, '.b', ms=2, label='rotor_speed-power')
ax.set_xlabel('RotorSpeed')
ax.set_ylabel('Power')
plt.legend(loc='upper right')

- 很明显，应该是有一个指数的曲线存在，曲线上方的就是异常点

## WindNumber - Power - WindSpeed

In [ ]:
sns.set_palette("Paired")
sns.set(rc={"figure.figsize": (25, 10)})
sns.relplot('WindSpeed','Power',hue='WindNumber', data=combined_data,kind='scatter', )